In [5]:
import heapq, random

class PriorityQueue:
    """
      Implements a priority queue data structure. Each inserted item
      has a priority associated with it and the client is usually interested
      in quick retrieval of the lowest-priority item in the queue. This
      data structure allows O(1) access to the lowest-priority item.
    """
    def  __init__(self):
        self.heap = []
        self.count = 0

    def push(self, item, priority):
        entry = (priority, self.count, item)
        heapq.heappush(self.heap, entry)
        self.count += 1

    def pop(self):
        (_, _, item) = heapq.heappop(self.heap)
        return item

    def isEmpty(self):
        return len(self.heap) == 0

    def update(self, item, priority):
        # If item already in priority queue with higher priority, update its priority and rebuild the heap.
        # If item already in priority queue with equal or lower priority, do nothing.
        # If item not in priority queue, do the same thing as self.push.
        for index, (p, c, i) in enumerate(self.heap):
            if i == item:
                if p <= priority:
                    break
                del self.heap[index]
                self.heap.append((priority, c, item))
                heapq.heapify(self.heap)
                break
        else:
            self.push(item, priority)

In [2]:
class Limb:
    def __init(self, name, strength, flexibility, hold):
        self.name, self.strength, self.flexibility, self.hold = name, strength, flexibility, hold

In [3]:
class Route:
    def __init__(self):
        self.holds = [(0, 1), (1, 1), (1, 3), (4, 3), (6, 7), (10, 11), (14, 12), (15, 15)]
        self.start_hold1 = (0, 1)
        self.start_hold2 = (1, 1)
        self.finish_hold = (15, 15)

In [4]:
class Hold:
    def __init__(self, coords, dif, width, height, angle):
        #Coords = top left corner
        self.coords = coords
        self.dif = dif
        self.width = width
        self.height = height
        self.angle = angle
    def getCenter(self):
        return (self.coords[0] + self.width/2, self.coords[1] + self.height/2)

In [5]:
class State:
    def __init__(self, lh: Limb, rh: Limb, lf: Limb, rf: Limb):
        self.lh, self.rh, self.lf, self.rf = lh, rh, lf, rf
    def eval_difficulty(self):
        #TODO: rewrite all with state class
        diff = 0
        diff += (0.5*abs(self.lh.x - self.rh.x) / 2 
                        - abs(self.lf.x - self.rf.x) / 2)
        #fix formula
        diff += (0.5*[(self.lf.y - self.rf.y) / 2 -
                   (self.lh.y - self.rh.y) / 2 + 60] ** 2)
        
        for limb in self.limbs:
            diff += limb.hold.diff / limb.strength
            #check out angles
            if limb == 'Left_Hand':
                if 75 > limb.hold.orientation > 0:
                    diff += 1.5
                elif 180 > limb.hold.orientation > 75:
                    diff += 1
                elif 270 > limb.hold.orientation > 180:
                    diff += 3
                else:
                    diff += 7
            #change angles to work with rh
            if limb == 'Right_Hand':
                if 75 > limb.hold.orientation > 0:
                    diff += 1.5
                elif 180 > limb.hold.orientation > 75:
                    diff += 1
                elif 270 > limb.hold.orientation > 180:
                    diff += 3
                else:
                    diff += 7
            #add something for underclings?

        # Arms stretched out too long top to bottom 
        if  abs(self.lh.y - self.rh.y) > 18:
            diff += 0.5 * abs(self.lh.y - self.rh.y) 
        # Arms stretched out too wide
        if  abs(self.lh.x - self.rh.x) > 55:
            diff += 0.5 * abs(self.lh.x - self.rh.x) 
        #Legs difficulty
        if abs(self.lf.y - self.rf.y) > 18:
            diff += 0.5 * abs(self.lf.y - self.rf.y)
        if abs(self.lf.x - self.rf.x) > 18:
            diff += 0.5 * abs(self.lf.x - self.rf.x)  
        return diff

In [34]:
import numpy as np
class aStar:
    WALL_HEIGHT = 180
    def __init__(self, route):
        self.route = route
#         self.holds = route.holds
        self.explored = []
        self.reach = 96 #from foot
        self.limbs = [Limbs.LEFT_HAND, Limbs.RIGHT_HAND, Limbs.LEFT_LEG, Limbs.RIGHT_LEG]        
    
    def getSuccessors(self, cur_hold, limb, state):
        successors = []

        for hold in self.route.holds:
            if (self.get_distance(hold, cur_hold) < self.reach and 
            self.eval_difficulty(limb, hold)[0] < 10 and hold not in self.explored):
                successors.append((hold, self.eval_difficulty(limb, cur_hold, hold, state)[1], 
                                   self.eval_difficulty(limb, cur_hold, hold, state)[0]))
#         print(successors[0][1])
        return successors
    
    def get_distance(self, hold, state):
        return np.sqrt((state[0] - hold[0]) ** 2 + (state[1] - hold[1]) ** 2)
    
    
        #return (1, "move")
    
    #def convert_pixel_to_inch(self):
        
    
#     def eval_action(self, state):
#         dict2 = {}
#         for limb in limbs:
#             for hold in self.route.holds:
#                 If dict1[limb].contains(hold):
#                     pass
#                 elif distance(limb.pos, hold.pos) >some number:
#                     pass
#                 else:
#                     difficulty = evaluation(action)
#                     dict2[(limb,hold)]= difficulty
#         return min(dict2)

    
    def uniformCostSearch(self):
        state = [None, None, None, None]
        l_dif = self.eval_difficulty(self.limbs[0], self.route.start_hold1)[0]
        r_dif = self.eval_difficulty(self.limbs[1], self.route.start_hold1)[0]
        frontier = PriorityQueue()

        if l_dif < r_dif:
            state = [self.route.start_hold1,self.route.start_hold2, None, None]
            cur_state1 = [state[0], [], l_dif]
            cur_state2 = [state[1], [], r_dif]
            frontier.push(cur_state1, l_dif)
            frontier.push(cur_state2, r_dif)
        else:
            state = [self.route.start_hold2, self.route.start_hold1, None, None]
            cur_state1 = [state[0], [], l_dif]
            cur_state2 = [state[1], [], r_dif]
            frontier.push(cur_state2, r_dif)
            frontier.push(cur_state1, l_dif)

        index = 2
        explored = {0:[], 1:[], 2:[], 3:[]}
        while not frontier.isEmpty():
            current_limb = self.limbs[index]
            cur_hold = frontier.pop()
            explored[index].append(cur_hold[0])
            state[index] = cur_hold[0]
            if cur_hold[0] == self.route.finish_hold:
                print(cur_hold[1])
                return cur_hold[1]
            for hold in self.getSuccessors(cur_hold[0], current_limb, state):
                next_cost = cur_hold[2] + hold[2]
                next_hold = [hold[0], cur_hold[1] + [hold[1]], next_cost]
                if hold[0] not in explored[index]: 
                    add_cond = True
                    for item in frontier.heap:
                        if item[2][0] == hold[0]:
                            if next_cost < item[2][2]: 
                                item[2][1] = next_hold[1]
                                frontier.update(item[2], next_cost)
                            add_cond = False
                            break
                    if add_cond:
                        frontier.push(next_hold, next_cost)
                        state[index] = next_hold
            index += 1 
            index = index % len(self.limbs)
#             print(index)


In [35]:
route = Route()
a_star = aStar(route)

print(a_star.uniformCostSearch())


TypeError: aStar.eval_difficulty() missing 2 required positional arguments: 'next_hold' and 'state'